In [1]:
import requests
import json
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
from database_utils import Database
import pandas as pd
import time
import datetime
import random
import sys

In [58]:
def LoadUserAgents(uafile):
    uas = []
    with open(uafile, 'rb') as uaf:
        for ua in uaf.readlines():
            if ua:
                uas.append(ua.strip()[:-1])
    random.shuffle(uas)
    return uas

uas = LoadUserAgents("user_agents.txt")

# 1. 根据关键词查出所有视频的相关信息

In [159]:
keyword="鸿蒙系统"
res=[]

In [160]:
# 按url获取当前页的视频列表
def get_video_list(url):
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    all_videos = soup.find_all(name='ul',attrs={"class":"video-contain clearfix"})[0]
    all_videos = all_videos.find_all(name='div', attrs={"class":"info"})
    return all_videos
# 从url中分离出up主id
def get_up_id(url):
    return url[url.index("com/") + 4: url.index("?")]
# 获取视频中的信息
def get_vider_data(source, res):
    for i, every_video in enumerate(source):
        data = {}
        data['av_id'] = every_video.find_all(name='span', attrs={"class":"type avid"})[0].text[2:]
        data['title'] = every_video.a['title']

        tags = every_video.find_all(name='div', attrs={"class":"tags"})[0]
        for j, s in enumerate(tags):
            data[s['title']] = s.text.strip()
            if j == 3:
                data['up_id'] = get_up_id(s.a['href'])
        res.append(data)

In [161]:
for page in range(1, 100):
    url = r'https://search.bilibili.com/all?keyword=' + keyword +'&page=' + str(page)
    video_list = get_video_list(url)
    if len(video_list) == 0:
        print("共%d页"% (page - 1))
        break
    else:
        get_vider_data(video_list, res)   

video_info = pd.DataFrame(data=res)

共12页


### 对视频信息进行补充

In [171]:
def get_video_other_info(row):
    time.sleep(0.7)
    oid = row['av_id']
    print(oid)
    head = {
    'User-Agent': random.choice(uas),
    'Connection': 'close'
    } 
    requests.adapters.DEFAULT_RETRIES =5
    curUrl = 'https://api.bilibili.com/x/web-interface/archive/stat?aid={}'.format(oid)
    source = requests.get(curUrl, headers=head, timeout=3000).text       
    source = json.loads(source)['data']
    row['view']     = source['view']
    row['reply']    = source['reply']
    row['favorite'] = source['favorite']
    row['coin']     = source['coin']
    row['share']    = source['share']
    row['like']     = source['like']
    return row

In [172]:
video_info = video_info.apply(get_video_other_info, axis=1)

53345246
53345246
54794918
52796568
54630655
55053731
54967488
55744212
55207724
53328041
55672253
56757076
54185448
55871847
54808443
55163080
54774617
54023949
54795414
56566386
56328662
55055300
55292820
56069095
55928221
55193955
53269704
56319288
56690345
54254071
55673800
54676161
54866488
56026601
53138350
55099100
54568170
56305719
53507788
56471171
54387187
55452932
53599380
56097737
54970383
55410825
53901142
53776538
54591729
56679613
55330428
55529530
54525537
55287087
53394202
56406980
53742146
56544044
56956208
55141678
54195256
54588950
52913414
55701353
55144339
56072599
56787788
53574842
53120333
53468186
55519802
56027856
53466033
55931507
56318955
53486173
55786882
53080582
54728227
53862284
55388409
54945286
54263517
53644520
53445588
56955057
53254439
54766664
55427841
53372133
53488396
55548427
56822874
55243270
54775666
56193149
55708153
55962072
53425999
55994986
54150596
55385129
54330157
55236206
53487830
53597561
54345100
54467191
55623466
53957934
55471531
5

In [173]:
video_info.sort_values('like', ascending=False)

,av_id,title,up_id,up主,上传时间,弹幕,观看,view,reply,favorite,coin,share,like
19,56328662,【科技袁人】华为的“鸿蒙”，能带领中国自主系统走多远？,419501714,科技袁人,2019-06-21,5217,19.0万,190518,2000,3700,9465,1320,11066
167,53819180,【美人鱼名场面】当Google知道华为自研操作系统后…,89422932,太宅夜,2019-05-28,165,11.9万,119336,420,1299,1618,319,3593
8,53328041,华为鸿蒙系统下的评论也太欢乐了吧,342195139,鸽中之王者,2019-05-23,375,65.3万,653497,2846,798,318,268,3061
55,53742146,华为鸿蒙能否取代安卓成5G操作系统一哥？,92119082,真像大白官方账号,2019-05-27,464,13.2万,132242,1524,613,894,139,2687
16,54023949,【军情】华为公布鸿蒙系统后：外国网友表示很欢迎,32693432,迷彩虎,2019-05-30,46,24.1万,241439,1039,395,243,105,2493
76,53080582,「领菁资讯」谷歌中断华为合作！华为自研系统曝光：鸿蒙 OS！/ 真香警告？苹果 iPhon...,10175373,领菁科技,2019-05-20,657,7.3万,73323,1599,210,337,65,1376
1,54794918,【华为鸿蒙系统界面曝光 】可能是手机中最好的操作系统，独立于安卓和iOS外,142450442,会说话的橙砸,2019-06-07,536,19.2万,192432,665,411,174,140,1292
83,53445588,「领菁资讯」箭在弦上！国产自研系统“华为鸿蒙”完成商标注册！/ 不讲究？网友吐槽一加 7 P...,10175373,领菁科技,2019-05-24,609,3.5万,34912,804,135,295,54,1114
119,53964706,「领菁资讯」“鸿蒙”系统已成？！华为注册“ARK OS”方舟系统商标！/ 致敬华为P30 P...,10175373,领菁科技,2019-05-29,282,3.3万,32572,474,105,237,40,1108
129,54625743,「领菁资讯」诺亚方舟现身！疑似华为自研鸿蒙 ARK OS 系统现身！/ 5G 已来！华为Ma...,10175373,领菁科技,2019-06-05,188,2.9万,28781,407,96,271,28,1104


In [174]:
video_info.to_csv("video_info.csv", index=False)

# 2. 爬取所有视频的弹幕

In [ ]:
barrages = []
av_id_list = video_info[video_info['弹幕'] != '0']['av_id'].tolist()

In [ ]:
def get_cid(avid):
    source = requests.get("https://api.bilibili.com/x/web-interface/view?aid=" + str(avid)).text
    source = json.loads(source)
    cid = source['data']['cid']
    return cid

In [ ]:
cols_name = ['video_time', 'barrage_type', 'font_size', 'font_color',
             'time', 'barrage_pool_type', 'sender_id', 'barrage_id']
def get_barrages(avid):
    curr_list = []
    cid = get_cid(avid)
    source = requests.get("https://comment.bilibili.com/"+ str(cid) +".xml")
    source.encoding = 'utf-8'
    source = BeautifulSoup(source.text, 'html.parser')
    source = source.find_all(name='d')
    for line in source:
        data = {'av_id': avid, 'content': line.text}
        for i, val in enumerate(line['p'].split(',')):
            data[cols_name[i]] = val
#     source = json.loads(source)
        curr_list.append(data)
    return curr_list

In [ ]:
for i, avid in enumerate(av_id_list):
    barrages += get_barrages(avid)

In [ ]:
barrages_csv = pd.DataFrame(data=barrages)
barrages_csv.to_csv("barrage_csv.csv", index=False)

In [168]:
barrages_csv

,av_id,barrage_id,barrage_pool_type,barrage_type,content,font_color,font_size,sender_id,time,video_time,u_id
0,53345246.0,16493091978477572,0,1,举报了，缺德up主,16777215.0,25.0,e7e91c0c,1.558600e+09,10.106,196631463
1,53345246.0,16497085347528708,0,1,豆奶视频 哈哈哈,16777215.0,25.0,719f2db2,1.558608e+09,82.665,360224268
2,53345246.0,16498341391106048,0,1,不是emui？,16777215.0,25.0,b784ebb7,1.558610e+09,48.012,225239456
3,53345246.0,16498404024647684,0,1,安卓,16777215.0,25.0,51a5ff8a,1.558610e+09,20.152,363902905
4,53345246.0,16498410431971328,0,1,广告真多,16777215.0,25.0,51a5ff8a,1.558610e+09,32.383,363902905
5,53345246.0,16498460921954304,0,1,边看变存,16777215.0,25.0,51a5ff8a,1.558610e+09,128.059,363902905
6,53345246.0,16498495281692672,0,1,这特么不是oneui么,16777215.0,25.0,a0fb36fa,1.558611e+09,2.989,96964692
7,53345246.0,16498796672319488,0,1,这是安卓9好吗！！！哗众取宠,16777215.0,25.0,21fa128d,1.558611e+09,60.502,6909276
8,53345246.0,16498879850610692,0,1,举报走一波,16777215.0,25.0,25c8bca3,1.558611e+09,1.010,44747257
9,53345246.0,16499496957509634,0,1,举报走一波。,16777215.0,25.0,2e6800d8,1.558612e+09,16.742,325870124


### 添加弹幕中的所有u_id

In [ ]:
def get_uid(x):
    res = database_util.get_uid(x)
    return res


In [ ]:
database_util = Database()
barrages_csv['u_id'] = barrages_csv['sender_id'].apply(lambda x: get_uid(x))
database_util.close()

In [ ]:
barrages_csv.to_csv("barrage_csv.csv", index=False)

# 3. 获取弹幕中所有用户的信息

In [2]:
barrages_csv = pd.read_csv("barrage_csv.csv")

In [3]:
barrages_csv = barrages_csv[barrages_csv['u_id'].notnull()]
barrages_csv['u_id'] = barrages_csv['u_id'].astype(int).astype(str)

In [25]:
def get_user_info(uid):
    
    head = {
    'User-Agent': random.choice(uas),
    'Connection': 'close',
    }
    data = {'u_id': uid}
    requests.adapters.DEFAULT_RETRIES =5
    source = requests.get("https://api.bilibili.com/x/space/acc/info?mid="+ str(uid) +"&jsonp=jsonp",\
                          timeout=3000)
    res = requests.get(
                'https://api.bilibili.com/x/relation/stat?vmid=' + str(uid) + '&jsonp=jsonp').text
    js_fans_data = json.loads(res)         
    source = json.loads(source.text)['data']
    data['name'] = source['name']
    data['sex'] = source['sex']
    data['sign'] = source['sign']
    data['level'] = source['level']
    data['birthday'] = source['birthday']
    data['vip'] = 1 if source['vip']['status'] == 1 else 0 
    data['following'] = js_fans_data['data']['following']
    data['fans'] = js_fans_data['data']['follower']
    return data

In [27]:
user_info_list=[]

In [29]:
user_list = barrages_csv['u_id'].tolist()
idx = -1
while idx < len(user_list):
    idx += 1
    
    if idx == len(user_list):
        break
        
    try:
        user_info_list.append(get_user_info(user_list[idx]))
    except:
        print("Connection refused by the server..and i is" + str(idx))
        time.sleep(5)
        idx -= 1
        continue

Connection refused by the server..and i is1199
Connection refused by the server..and i is1199
Connection refused by the server..and i is1200
Connection refused by the server..and i is1202
Connection refused by the server..and i is1204
Connection refused by the server..and i is1204
Connection refused by the server..and i is1204
Connection refused by the server..and i is1205
Connection refused by the server..and i is1205
Connection refused by the server..and i is3482
Connection refused by the server..and i is3496
Connection refused by the server..and i is3661
Connection refused by the server..and i is4445
Connection refused by the server..and i is4533
Connection refused by the server..and i is5050
Connection refused by the server..and i is5562
Connection refused by the server..and i is6085
Connection refused by the server..and i is6730
Connection refused by the server..and i is7617
Connection refused by the server..and i is7617
Connection refused by the server..and i is7620
Connection re

In [83]:
user_info = pd.DataFrame(data=user_info_list).drop_duplicates('u_id')
user_info.to_csv('user_info.csv', index=False)